In [10]:
# imports

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import scipy
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [11]:
# three types of fake data for testing

# def signal1(A, t):
#     return A * np.sin(30*t) * np.exp(-t) + np.random.randn(t.shape[0])

# def signal2(A, t):
#     return A * np.sin(30.5*t) * np.exp(-t)+ np.random.randn(t.shape[0])

# def signal3(A, t):
#     return A * np.sin(31*t) * np.exp(-t) + np.random.randn(t.shape[0])

In [12]:
# tests
A = 2
# t = np.linspace(0.0001, 5, 40000)
# s1 = signal1(A, t)
# s2 = signal2(A, t)
# s3 = signal3(A, t)

In [13]:
# plot fn for convenience
def plot(x, y, title):
    plt.figure()
    plt.plot(x, y, linewidth=0.7, c='blue')
    plt.title(title)
    plt.savefig(title + '.png')
    plt.show()

# plot(t, s1, 'Example1')
# plot(t, s2, 'Example2')
# plot(t, s3, 'Example3')


In [14]:
# # Number of samples and length of each signal
# n_samples = 200
# signal_length = 40000

# # Initialize arrays
# X_data = np.zeros((n_samples, signal_length))
# y_labels = np.zeros(n_samples)

# # time array
# t = np.linspace(0, 1, signal_length)

# # Generate samples
# for i in range(n_samples):
#     if i < n_samples // 3:
#         X_data[i] = signal1(A, t)
#         y_labels[i] = 0
#     elif i < 2 * n_samples // 3:
#         X_data[i] = signal2(A, t)
#         y_labels[i] = 1
#     else:
#         X_data[i] = signal3(A, t)
#         y_labels[i] = 2

# # One-hot encode the labels
# y_labels_one_hot = to_categorical(y_labels, num_classes=3)

In [47]:
test_filename = 'xa.s12.00.mhz.1970-03-25HR00_evid00003_trimmed_7000_sec'

data_directory = './'
mseed_file = f'{data_directory}{test_filename}.mseed'
st = read(mseed_file)
st

tr = st.traces[0].copy()
tr_times = tr.times()
tr_data = tr.data

# plot(tr_times, tr_data, 'Mseed Example')

# print(tr_times.shape)

def read_all_mseed_files(data_directory, target_length=None):
    # List all files in the directory with ".mseed" extension
    mseed_files = [f for f in os.listdir(data_directory) if f.endswith('.mseed')]
    
    data_matrix = []
    
    # Loop through all the mseed files and extract time and data series
    for filename in mseed_files:
        st = read(os.path.join(data_directory, filename))
        tr = st.traces[0].copy()  
        tr_data = tr.data  
        
        if target_length is None:
            target_length = len(tr_data)  # Set target length to the first trace's length
        
        # Pad or trim the data to the target length
        if len(tr_data) < target_length:
            # Pad with zeros if shorter
            tr_data = np.pad(tr_data, (0, target_length - len(tr_data)), mode='constant')
        else:
            # Trim if longer
            tr_data = tr_data[:target_length]
        
        data_matrix.append(tr_data)
    
    # Convert the list to a numpy matrix
    data_matrix = np.array(data_matrix)
    
    return data_matrix

X_data = read_all_mseed_files(data_directory, 46376)

# plot(tr_times, data[3], 'Example 3')

In [48]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# read csv
df = pd.read_csv('catalog.csv', header=None, names=['data'], skiprows=1)

# split cells
df['label'] = df['data'].apply(lambda x: x.split(',')[-1].strip())

# Step 3: One-hot encode the labels
encoder = OneHotEncoder(sparse_output=False)
y_label = encoder.fit_transform(df['label'].values.reshape(-1, 1))



y_label[4]

# one-hot format: [deep, impact, shallow]

array([1., 0., 0.])

In [49]:
input_shape = (46376,)

# Build the classifier model
def build_classifier(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Dense(256, activation='relu')(inputs)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(64, activation='relu')(x)
    
    # Output layer (one hot encoding)
    outputs = layers.Dense(3, activation='softmax')(x)
    
    model = models.Model(inputs, outputs)
    return model

classifier = build_classifier(input_shape)

# Compile the model
classifier.compile(optimizer='adam', 
                   loss='categorical_crossentropy', 
                   metrics=['CategoricalAccuracy'])

# Display model
classifier.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 46376)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │    11,872,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,913,859 (45.45 MB)

 Trainable params: 11,913,859 (45.45 MB)

 Non-trainable params: 0 (0.00 B)

In [50]:
# training
classifier.fit(X_data, y_label, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 374ms/step - CategoricalAccuracy: 0.3743 - loss: 1.0963 - val_CategoricalAccuracy: 0.8750 - val_loss: 1.0815
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - CategoricalAccuracy: 0.8576 - loss: 1.0794 - val_CategoricalAccuracy: 0.8750 - val_loss: 1.0604
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - CategoricalAccuracy: 0.8160 - loss: 1.0603 - val_CategoricalAccuracy: 0.8750 - val_loss: 1.0354
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - CategoricalAccuracy: 0.8576 - loss: 1.0334 - val_CategoricalAccuracy: 0.8750 - val_loss: 1.0057
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - CategoricalAccuracy: 0.8264 - loss: 1.0080 - val_CategoricalAccuracy: 0.8750 - val_loss: 0.9703
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - CategoricalAccuracy: 0.8264 - loss: 0.9747 - val_CategoricalAccuracy: 0.8750 - val_loss: 0.9285
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - CategoricalAccuracy: 0.8264 - loss: 0.9361 - val_CategoricalA

In [62]:
# test

test = X_data[8]
test = test.reshape(1, -1)

# Now call predict
prediction = classifier.predict(test)

print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[0.23078983 0.55988955 0.20932065]]
